In [14]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import pandas as pd
import yaml
from tqdm import tqdm
import cv2

In [16]:
img_width, img_height = 150, 150
image_size = (img_width, img_height)

In [19]:
df_train = pd.read_csv('../data/train_v2.csv')

with open('../label_maps.yml', 'r') as lablels:
    label_data = yaml.load(lablels)
inv_label_map = label_data['inv_label_map']
label_map = label_data['label_map']

x_train = []
y_train = []

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('../data/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, image_size))
    y_train.append(targets)
    
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.uint8)

100%|██████████| 40479/40479 [01:05<00:00, 614.42it/s]


In [20]:
import numpy as np
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.15, random_state=42)
print(x_train.shape)
print(y_train.shape)

(34407, 150, 150, 3)
(34407, 17)


In [59]:
x_train = x_train[:10]
x_valid = x_valid[:5]
y_train = y_train[:10]
y_valid = y_valid[:5]
nb_train_samples = len(x_train)
nb_validation_samples = len(x_valid)
epochs = 50
batch_size = 1

In [52]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    print('Loading VGG16 model')
    model = applications.VGG16(include_top=False, weights='imagenet')

    # generator dose not shuffle images, the output should be in the same order as the data
    print('Starting process training data')
    generator = datagen.flow(
        x_train,
        batch_size=batch_size,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size, verbose = 1)
    print(bottleneck_features_train.dtype)
    np.save(open('../models/bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)
    
    print('Starting process validation data')
    generator = datagen.flow(
        x_valid,
        batch_size=batch_size,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size, verbose = 1)
    np.save(open('../models/bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)
    

In [53]:
save_bottlebeck_features()

Loading VGG16 model
Starting process training data
10/10 [==============================] - 1s     
float32
Starting process validation data
5/5 [==============================] - 0s     


In [60]:
def train_top_model():
    train_data = np.load(open('../models/bottleneck_features_train.npy', 'rb'))
    train_labels = y_train

    validation_data = np.load(open('../models/bottleneck_features_validation.npy', 'rb'))
    validation_labels = y_valid

    model = Sequential()
    print(train_data.shape[1:])
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [61]:
train_top_model()

(4, 4, 512)
Train on 10 samples, validate on 5 samples
Epoch 1/50
10/10 [==============================] - 0s - loss: 0.4748 - acc: 0.8294 - val_loss: 0.6803 - val_acc: 0.8353
Epoch 2/50
10/10 [==============================] - 0s - loss: 0.3495 - acc: 0.8882 - val_loss: 0.7271 - val_acc: 0.8353
Epoch 3/50
10/10 [==============================] - 0s - loss: 0.2648 - acc: 0.9000 - val_loss: 0.6961 - val_acc: 0.8235
Epoch 4/50
10/10 [==============================] - 0s - loss: 0.3162 - acc: 0.9000 - val_loss: 0.6686 - val_acc: 0.8353
Epoch 5/50
10/10 [==============================] - 0s - loss: 0.2866 - acc: 0.9118 - val_loss: 0.6356 - val_acc: 0.8471
Epoch 6/50
10/10 [==============================] - 0s - loss: 0.2093 - acc: 0.9353 - val_loss: 0.6750 - val_acc: 0.8353
Epoch 7/50
10/10 [==============================] - 0s - loss: 0.1999 - acc: 0.9235 - val_loss: 0.8292 - val_acc: 0.8118
Epoch 8/50
10/10 [==============================] - 0s - loss: 0.2321 - acc: 0.9059 - val_loss: 0.